# Make Bag-of-visual-words

* Load Image's descriptor for all images. 
* Find corresponding visual word index for each descriptor. 
* Save list of visual word index.

# For Oxford 5k dataset, You can use already provided visual words 

In [17]:
%%time
# Oxford 5k dataset provides already converted visual words. We could use this one
import os 
import pickle

oxf5k_visualword_dir = './data/word_oxc1_hesaff_sift_16M_1M'
work_dir = "./oxfk5_provided"
if not os.path.exists(work_dir):
    os.mkdir(work_dir)

filelist = os.listdir(oxf5k_visualword_dir)
filelist.sort()
# print(filelist)
# for parent_dir, _, files in os.walk(oxf5k_visualword_dir):
#     print(files)

bow_dict = {}
count_descriptor = 0
image_feature_count_info = []
for filename in filelist:
    filepath = os.path.join(oxf5k_visualword_dir, filename)
    image_name = filename.replace(".txt", "")    
    visual_words = []
    with open(filepath) as f:
        lines = list(map(lambda x: x.strip(), f.readlines()[2:])) # ignore first two lines        
        for l in lines:
            val = l.split(" ")
            visual_word_index = int(val[0])-1 # This data use 1 to 1,000,000. convert to zero-based so 0 to 999,999  
            visual_words.append(visual_word_index)
            # print('{} descriptor {}'.format(filename, l))
        count_descriptor = count_descriptor + len(lines)
    image_feature_count_info.append((image_name, len(visual_words)))
    bow_dict[image_name] = sorted(visual_words)
    # break
# print('bow_dict:', bow_dict)
print('count_descriptor:', count_descriptor)

pickle.dump(bow_dict, open(os.path.join(work_dir, 'bow_dict_word_oxc1_hesaff_sift_16M_1M_pretrained.pkl'), 'wb'))


count_descriptor: 16334970


# Or we could make a new one from provided feature info for Oxford 5k

Oxford 5k dataset already provide SIFT descriptors, and visual words info. 
The file containing SIFT descriptor does not have information of how many descriptors are belong to which image. 
This missing information can be found in visual words info file. 
After we get the assignment relationship, we can successfully get dictionary of key: image name, values: list of descriptos.

## Requirements

* SIFT descriptor containing file. `feat_oxc1_hesaff_sift.bin`
* Image filename order for the above file `order.txt`
* Bag-of-words informatino file `word_oxc1_hesaff_sift_16M_1M`


In [ ]:
# For oxford 5k dataset, it may be possible to recover the "Image name and its associated 128d descriptors"
from utils.oxf5k_feature_reader import feature_reader

# Frist, read all 128d descriptor of image.
feature_bin_path = "./data/feature/feat_oxc1_hesaff_sift.bin"
all_features = feature_reader(feature_bin_path)
print('num of 128d descriptors in oxf5k: ', len(all_features))


In [ ]:
# Read the order of filenames related to above features. Refer README in http://www.robots.ox.ac.uk/~vgg/data/oxbuildings/README2.txt
import os
feature_bin_filename_order_path = "./data/feature/order.txt"
with open(feature_bin_filename_order_path) as f:
    filenames = list(map(lambda x: x.strip(), f.readlines()))

BOW_INFO_DIR='./data/word_oxc1_hesaff_sift_16M_1M'
image_feature_count_info = []
for image_name in filenames:
    filename = image_name + ".txt"
    with open(os.path.join(BOW_INFO_DIR, filename)) as f:
        header_text = list(map(lambda x: x.strip(), f.readlines()[:2]))        
        num_descriptor = int(header_text[1])        
        image_feature_count_info.append((image_name, num_descriptor))    
    
print('image_feature_count_info[:5]:',image_feature_count_info[:5])

# Check compatibility of name order
for idx, val in enumerate(image_feature_count_info):
    name_from_vw, _ = val
    if name_from_vw == filenames[idx]:
        continue
    else:
        print(idx, name_from_vw, filenames[idx])
print("compatibility check done")    

In [ ]:
import numpy as np
import pickle

image_descriptor_dict = {} # key: image_name, value: 2d numpy array of shape (num_descriptor, dim_descriptor)
start_idx = 0
for image_name, num_descriptor in image_feature_count_info:
    val = np.array(all_features[start_idx:(start_idx+num_descriptor)], dtype=np.uint8)
    image_descriptor_dict[image_name] = val
    
    start_idx += num_descriptor
    # break
# print('image_descriptor_dict:', image_descriptor_dict)
with open('image_descriptor_dict_oxc1_hesaff_sift_16M.pkl', 'wb') as f:
    pickle.dump(image_descriptor_dict, f)

# After you prepared everything, run below code

## Preparation list

* (image, descriptor_list) tuple
* centroids

TODO: determine design choice when we use encoding such as Product Quantization. 
1. keep centroid as PQ code. It means we always have to keep encoder parts. 
2. keep centroid as original vector. It means there is room for additional error when we assign each descriptor to the nearest centroid. 

In [ ]:
# Load image descriptor dictionary, and assign each descriptor to visual words
import pickle
import numpy as np
import pqkmeans
from tqdm import tqdm
from multiprocessing import Pool, TimeoutError

image_descriptor_dict_path = 'image_descriptor_dict_oxc1_hesaff_sift_16M.pkl'

# You should use matching encoder that was used to do PQk-means clustering. 
# encoder_save_path = 'pqencoder_100k_random_sample_from_16M.pkl'
# cluster_center_save_path = 'clustering_centers_in_pqcode_numpy.npy'

# encoder_save_path = 'pqencoder_1000k_random_sample_from_16M.pkl'
# cluster_center_save_path = 'clustering_centers_numpy_16M_feature_1000k_coodebook_131k_cluster.npy'

encoder_save_path = 'pqencoder_1000k_8_sub_random_sample_from_16M.pkl'
cluster_center_save_path = 'clustering_centers_numpy_16M_feature_1000k_coodebook_8_sub_131k_cluster.npy'

output_bow_dict_save_path = 'bow_dict_word_oxc1_hesaff_sift_16M_1M_8_sub_handmade.pkl'

# encoder_save_path = 'encoder.pkl'
# cluster_center_save_path = 'clustering_centers_numpy.npy'
    
# For PQ-kmeans clustering, we first convert query to PQ codes. 
with open(encoder_save_path, 'rb') as f:
    encoder = pickle.load(f)
clustering_centers_in_pqcode_numpy = np.load(cluster_center_save_path)
# print('cluster centers shape: ', clustering_centers_in_pqcode_numpy.shape)

k = clustering_centers_in_pqcode_numpy.shape[0]
print('number of clusters:', k)
engine = pqkmeans.clustering.PQKMeans(encoder=encoder, k=k, iteration=1, verbose=False, init_centers=clustering_centers_in_pqcode_numpy)
    
bow_dict = {}


def run(val):
    image_name, descriptors = val
    data_points_pqcodes = encoder.transform(descriptors)
    # print('num_descriptors:', len(data_points_pqcodes))
    # print('num_descriptors shape:', data_points_pqcodes.shape)
    # TODO: speedup by using pq-kmeans assignment step. 
    # visual_words = get_assigned_center_index(data_points_pqcodes, clustering_centers_in_pqcode_numpy)
    visual_words = engine.predict(data_points_pqcodes) # Fast assignment step. 
    
    return (image_name, list(set(list(visual_words))))
    
if __name__ == "__main__":    
    
    with open(image_descriptor_dict_path, 'rb') as f:
        # key: image_name, value: 2d numpy array of shape (num_descriptor, dim_descriptor)
        image_descriptor_dict = pickle.load(f) 
    # print('num images:', len(image_descriptor_dict))
    
    pool = Pool(processes=20) # start 20 worker processes
    # print same numbers in arbitrary order
    for image_name, bow in tqdm(pool.imap_unordered(run, image_descriptor_dict.items()), total=len(image_descriptor_dict)):        
        bow_dict[image_name] = bow
        
    print('done')
    pickle.dump(bow_dict, open(output_bow_dict_save_path, 'wb'))
    
# Timing: 24min for 5062 images with 16M features. 100k leanred codebook with 4 subspaces. 2^17 clusters. 
# Timing: 38min for 5062 images with 16M features. 1M leanred codebook with 8 subspaces. 2^17 clusters. 

# Or we could make a new one

## Requirements

* Image name and its associated 128d descriptors
* visual words assigner
    * For a given 128d descriptor, it tells index of visual words. 
    * After you ran k-means clustering, you could use it to get nearest centroid's id
